### Description:
Explore and clean lab variables

- First grab by lab_names (or base_names), then check their base_names (or lab_names), from all_labs file (labs that we went through to select)
- Then reverse to include lab_names with more than 1 base_names and base_names with more than 1 lab_names.
    - Based off Triage work of selecting lab_names, take those final lab_names and get their base_names
    - Add a few more base_names 
    - Use these base_names to run m6_labs.sql. This makes sure we have everything from Triage selection.
- Check the units and ranges of these labs to keep only valid ones.
- Combine lab_names/base_names and convert to same units as needed --> rename these groups
- Most urine related labs are all NA

It would be shorter to use base_name, but still need to check because some same labs have different base_names

- For ord_num_value = 9999999 of extreme cases on either end (hi/lo, panic hi/lo):
    - Remove those observations where no < > in ord_value: cancelled or N/A
    - Use ord_value to take bounds values. For example, if ord_value is <0.2 and ord_num_value is 9999999, then replace 9999999 with 0.2

Output file: 
- `2_4_coh3_labs`
- `cohort3L_withlabs`: cohort information only from 2_4_coh3_labs

### Importing R libraries

In [2]:
library(bigrquery)  # to query STARR-OMOP (stored in BigQuery) using SQL
library(tidyverse)
library(lubridate)
# library(mice)
# library(VIM) # for missing data plot

# library(data.table)
# library(Matrix)
# library(caret) # import this before glmnet to avoid rlang version problem
# library(glmnet)
# library(bit64)

# library(slam)
# library(mtools) for one hot coder, not available on Nero or use caret or tidyr
options(repr.matrix.max.rows=250, repr.matrix.max.cols=30)

### Set up and run queries
Do this on Nero GCP as querying from a local computer takes much longer time

In [3]:
# CREDENTIALS depending on LOCATIONS:
# credential <- paste0("/home/", "minh084", "/.config/gcloud/application_default_credentials.json")

# local computer
# credential <- "C:/Users/User/AppData/Roaming/gcloud/application_default_credentials.json"

# Nero onprem
# credential <- "/home/minh084/.config/gcloud/application_default_credentials.json"

# Nero gcp notebook
credential <- "/home/jupyter/.config/gcloud/application_default_credentials.json"

project_id <- "som-nero-phi-jonc101"

Sys.setenv(GOOGLE_APPLICATION_CREDENTIALS = credential)
Sys.setenv(GCLOUD_PROJECT = project_id)
gargle::credentials_app_default()

NULL

In [4]:
library(DBI)
con <- dbConnect(
  bigrquery::bigquery(),
  project = project_id,
  dataset = "shc_core" #, billing = project_id
)
con 
dbListTables(con)

<BigQueryConnection>
  Dataset: som-nero-phi-jonc101.shc_core
  Billing: som-nero-phi-jonc101

[1] "adt"                 "allergy"             "culture_sensitivity"
 [4] "demographic"         "dep_map"             "diagnosis_code"     
 [7] "drg_code"            "encounter"           "family_hx"          
[10] "flowsheet"           "lab_result"          "lda"                
[13] "mapped_meds"         "mar"                 "med_orderset"       
[16] "ndc_code"            "order_med"           "order_proc"         
[19] "order_quest"         "proc_note"           "proc_orderset"      
[22] "procedure_code"      "prov_map"            "social_hx"          
[25] "treatment_team"

### LABS
1. Use lab results as is. no manually process as with vital signs. 
2. Some notes about labs:
- Cannot trust "result_in_range_yn" --> remove this
- There is 1 incident of Glucose by Meter = hi in ord_value
- ord_value has text, blanks and #
- ord_num_value has NA, 9999999, #. The blanks from ord_value are NA in ord_num_value
- result_flag: "Abnormal" for text in ord_value, usually correct. However, when ord_value is N/A or < or >, the ord_num_value is 9999999.
3. Use ord_num_value and result_flag are good
4. All the urine labs start with X seem to be NA. Remove mag and phos as these are low count 1K

In [115]:
# read inputs
datadir = "../../DataTD"
cohortdir = "../../OutputTD/1_cohort"
featuredir = "../../OutputTD/2_features"

cohort <- read.csv(file.path(cohortdir, "1_3_cohort.csv"))
labs0 <- read.csv(file.path(datadir, "labs.csv")) # 1386585

# check for unique CSNs and MRNs
nrow(cohort) #44258
nrow(labs0) #depending on which cohort_demo used to query labs 1863768
nrow(labs0 %>% select(pat_enc_csn_id_coded) %>% distinct()) # 41724 vs 41627
colnames(labs0)

[1] 44258

[1] 1372519

[1] 41627

[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "inpatient_data_id_coded" "admit_time"             
 [5] "label"                   "DBP"                    
 [7] "Pulse"                   "RR"                     
 [9] "SBP"                     "Temp"                   
[11] "order_id_coded"          "lab_name"               
[13] "base_name"               "ord_value"              
[15] "ord_num_value"           "reference_low"          
[17] "reference_high"          "reference_unit"         
[19] "result_in_range_yn"      "result_flag"            
[21] "result_time_utc"         "order_time_utc"         
[23] "taken_time_utc"

In [116]:
labs0 <- labs0 %>% mutate(reference_unit = factor(reference_unit),
                          reference_low = factor(reference_low),
                          reference_high = factor(reference_high),
                          result_in_range_yn = factor(result_in_range_yn),
                          result_flag = factor(result_flag)) #%>% 

In [118]:
options(repr.matrix.max.rows=200, repr.matrix.max.cols=20)
# labs0 %>% group_by(base_name, lab_name) %>% count() %>% arrange(base_name)
labs0 %>% group_by(base_name, lab_name) %>% count() %>% arrange(lab_name)

base_name,lab_name,n
<chr>,<chr>,<int>
ALB,"Albumin, Ser/Plas",38332
ALKP,"Alk P'TASE, Total, Ser/Plas",38223
ALT,"ALT (SGPT), Ser/Plas",38221
AG,Anion Gap,42004
AGAP,"Anion Gap, ISTAT",2592
PCAGP,"Anion Gap, ISTAT",155
PCO2A,Arterial pCO2 for POC,12
PHA,Arterial pH for POC,12
PO2A,Arterial pO2 for POC,12


In [119]:
labs0 %>% group_by(base_name) %>% count(sort=TRUE)

base_name,n
<chr>,<int>
GLU,52035
HCT,45749
K,45742
HGB,45719
NA,45667
CR,45093
CL,44930
BUN,44584
PLT,42318


In [120]:
# from Triage
triage_labs <- c("Platelet count", "Total Bilirubin, Ser/Plas", "Total Bilirubin", 
"TROPONIN I", "Troponin I, POCT", "WBC", "WBC count", 
"Hematocrit", "Hct, ISTAT", "Hct(Calc), ISTAT", 
"Hct (Est)", "HCT, POC", 
"Hematocrit (Manual Entry) See EMR for details", 
"Hemoglobin", "Hgb(Calc), ISTAT", "Hgb, calculated, POC", "HgB", 
"Potassium, Ser/Plas", "Potassium, ISTAT",  "Potassium, Whole Bld", 
"Potassium, whole blood, ePOC", "Sodium, Ser/Plas", "Sodium, ISTAT",  
"Sodium, Whole Blood", "Chloride, Ser/Plas",  "Chloride, ISTAT",  
"Chloride, Whole Bld", "Creatinine, Ser/Plas", "Creatinine,ISTAT", 
"Anion Gap", "Anion Gap, ISTAT", "Glucose, Ser/Plas", "Glucose,ISTAT", 
"Glucose, Whole Blood", "Glucose by Meter", "Glucose, Non-fasting", 
"BUN, Ser/Plas", "BUN, ISTAT", "Urea Nitrogen,Ser/Plas", 
"Neutrophil, Absolute", "NEUT, ABS", "Neut, ABS (Seg+Band) (man diff)", 
"Neutrophils, Absolute (Manual Diff)", "Neut, ABS (Seg+Band) (man diff)", 
"Basophil, Absolute", "BASOS, ABS", "Basophils, ABS (man diff)", 
"Baso, ABS (man diff)", "Eosinophil, Absolute", "EOS, ABS", 
"Eosinophils, ABS (man diff)", "Eos, ABS (man diff)", 
"Lymphocyte, Absolute", "LYM, ABS", "Lymphocytes, ABS (man diff)", 
"Lym, ABS (man diff)", "Lymphocytes, Abs.", "Monocyte, Absolute", 
"MONO, ABS", "Monocytes, ABS (man diff)", "Mono, ABS (man diff)", 
"Lactate, ISTAT", "Lactate, Whole Bld", "Lactic Acid", "HCO3", 
"HCO3 (a), ISTAT", "Bicarbonate, Art for POC", "HCO3 (v), ISTAT", 
"HCO3, ISTAT", "O2 Saturation (a)", "O2 Saturation, ISTAT", 
"Oxygen Saturation for POC", "O2 Saturation (v)",  
"O2 Saturation, ISTAT (Ven)", "pCO2 (a)", "pCO2 (a), ISTAT",  
"PCO2, ISTAT", "Arterial pCO2 for POC", "pCO2 (v)", "PCO2 (v), ISTAT", 
"pH (a)", "PH (a), ISTAT", "pH by Meter", "Arterial pH for POC", 
"pH (v)", "PH (v), ISTAT", "PH, ISTAT", "pO2 (a)", "PO2 (a), ISTAT",  
"PO2, ISTAT", "Arterial pO2 for POC", "pO2 (v)", "PO2 (v), ISTAT", 
"tCO2", "TCO2 (a), ISTAT","TCO2, ISTAT", "TCO2, (ISTAT)", 
"CO2 Arterial Total for POC", "INR", "INR, ISTAT", 
"Prothrombin Time", "PT, ISTAT")
str_sort(triage_labs)

[1] "Anion Gap"                                    
  [2] "Anion Gap, ISTAT"                             
  [3] "Arterial pCO2 for POC"                        
  [4] "Arterial pH for POC"                          
  [5] "Arterial pO2 for POC"                         
  [6] "Baso, ABS (man diff)"                         
  [7] "Basophil, Absolute"                           
  [8] "Basophils, ABS (man diff)"                    
  [9] "BASOS, ABS"                                   
 [10] "Bicarbonate, Art for POC"                     
 [11] "BUN, ISTAT"                                   
 [12] "BUN, Ser/Plas"                                
 [13] "Chloride, ISTAT"                              
 [14] "Chloride, Ser/Plas"                           
 [15] "Chloride, Whole Bld"                          
 [16] "CO2 Arterial Total for POC"                   
 [17] "Creatinine, Ser/Plas"                         
 [18] "Creatinine,ISTAT"                             
 [19] "EOS, ABS"                                     
 [20] "Eos, ABS (man diff)"                          
 [21] "Eosinophil, Absolute"                         
 [22] "Eosinophils, ABS (man diff)"                  
 [23] "Glucose by Meter"                             
 [24] "Glucose, Non-fasting"                         
 [25] "Glucose, Ser/Plas"                            
 [26] "Glucose, Whole Blood"                         
 [27] "Glucose,ISTAT"                                
 [28] "HCO3"                                         
 [29] "HCO3 (a), ISTAT"                              
 [30] "HCO3 (v), ISTAT"                              
 [31] "HCO3, ISTAT"                                  
 [32] "Hct (Est)"                                    
 [33] "Hct, ISTAT"                                   
 [34] "HCT, POC"                                     
 [35] "Hct(Calc), ISTAT"                             
 [36] "Hematocrit"                                   
 [37] "Hematocrit (Manual Entry) See EMR for details"
 [38] "Hemoglobin"                                   
 [39] "HgB"                                          
 [40] "Hgb, calculated, POC"                         
 [41] "Hgb(Calc), ISTAT"                             
 [42] "INR"                                          
 [43] "INR, ISTAT"                                   
 [44] "Lactate, ISTAT"                               
 [45] "Lactate, Whole Bld"                           
 [46] "Lactic Acid"                                  
 [47] "LYM, ABS"                                     
 [48] "Lym, ABS (man diff)"                          
 [49] "Lymphocyte, Absolute"                         
 [50] "Lymphocytes, ABS (man diff)"                  
 [51] "Lymphocytes, Abs."                            
 [52] "MONO, ABS"                                    
 [53] "Mono, ABS (man diff)"                         
 [54] "Monocyte, Absolute"                           
 [55] "Monocytes, ABS (man diff)"                    
 [56] "NEUT, ABS"                                    
 [57] "Neut, ABS (Seg+Band) (man diff)"              
 [58] "Neut, ABS (Seg+Band) (man diff)"              
 [59] "Neutrophil, Absolute"                         
 [60] "Neutrophils, Absolute (Manual Diff)"          
 [61] "O2 Saturation (a)"                            
 [62] "O2 Saturation (v)"                            
 [63] "O2 Saturation, ISTAT"                         
 [64] "O2 Saturation, ISTAT (Ven)"                   
 [65] "Oxygen Saturation for POC"                    
 [66] "pCO2 (a)"                                     
 [67] "pCO2 (a), ISTAT"                              
 [68] "pCO2 (v)"                                     
 [69] "PCO2 (v), ISTAT"                              
 [70] "PCO2, ISTAT"                                  
 [71] "pH (a)"                                       
 [72] "PH (a), ISTAT"                                
 [73] "pH (v)"                                       
 [74] "PH (v), ISTAT"                                
 [75] 

In [121]:
# labs0[labs0$lab_name == "Platelets", ]$base_name
# labs0[labs0$lab_name %in% lab_names, ]$base_name   
# lapply(lab_names, function(x) labs0[labs0$lab_name == x, ]$basename)

In [122]:
# check overlapping of Triage labs and the new queried labs
# str_sort(unique(labs0$base_name))
# str_sort(intersect(labs0$lab_name, lab_names))

setdiff(triage_labs, labs0$lab_name) # NONE
str_sort(setdiff(labs0$lab_name, triage_labs)) # in A, not B

triage_labs0_names <- labs0 %>% filter(lab_name %in% triage_labs) %>% distinct(lab_name, base_name)
triage_labs0_base_names <- str_sort(unique(triage_labs0_names$base_name))
triage_labs0_lab_names <- str_sort(unique(triage_labs0_names$lab_name))

triage_labs0_base_names; # triage_labs0_lab_names

str_sort(setdiff(labs0$base_name, triage_labs0_base_names))

character(0)

[1] "Albumin, Ser/Plas"            "Alk P'TASE, Total, Ser/Plas" 
 [3] "ALT (SGPT), Ser/Plas"         "AST (SGOT), Ser/Plas"        
 [5] "Base Excess (vt)"             "Base Excess Arterial for POC"
 [7] "Base Excess, ISTAT"           "Calcium, Ser/Plas"           
 [9] "CO2, Ser/Plas"                "eGFR"                        
[11] "Globulin"                     "Glucose"                     
[13] "MCH"                          "Neut, ABS (man diff)"        
[15] "Platelets"                    "Potassium"                   
[17] "Protein, Total, Ser/Plas"     "RDW"                         
[19] "Sodium, whole blood, ePOC"

[1] "AG"      "AGAP"    "BASOAB"  "BUN"     "CL"      "CR"      "EOSAB"  
 [8] "GLU"     "HCO3"    "HCO3A"   "HCO3V"   "HCT"     "HGB"     "INR"    
[15] "K"       "LAC"     "LACWBL"  "LYMAB"   "MONOAB"  "NEUTAB"  "NEUTABS"
[22] "O2SATA"  "O2SATV"  "PCAGP"   "PCBUN"   "PCCL"    "PCO2A"   "PCO2V"  
[29] "PH"      "PHA"     "PHV"     "PLT"     "PO2A"    "PO2V"    "PT"     
[36] "TBIL"    "TCO2A"   "TNI"     "WBC"     NA

[1] "ALB"  "ALKP" "ALT"  "AST"  "BE"   "CA"   "CO2"  "EGFR" "GLOB" "MCH" 
[11] "RDW"  "TP"

In [123]:
# get lab_names and base_names from labs0
labs0_names <- labs0 %>% distinct(lab_name, base_name)
base_names <- str_sort(unique(labs0_names$base_name))
lab_names <- str_sort(unique(labs0_names$lab_name))

lab_names

[1] "Albumin, Ser/Plas"                            
  [2] "Alk P'TASE, Total, Ser/Plas"                  
  [3] "ALT (SGPT), Ser/Plas"                         
  [4] "Anion Gap"                                    
  [5] "Anion Gap, ISTAT"                             
  [6] "Arterial pCO2 for POC"                        
  [7] "Arterial pH for POC"                          
  [8] "Arterial pO2 for POC"                         
  [9] "AST (SGOT), Ser/Plas"                         
 [10] "Base Excess (vt)"                             
 [11] "Base Excess Arterial for POC"                 
 [12] "Base Excess, ISTAT"                           
 [13] "Baso, ABS (man diff)"                         
 [14] "Basophil, Absolute"                           
 [15] "Basophils, ABS (man diff)"                    
 [16] "BASOS, ABS"                                   
 [17] "Bicarbonate, Art for POC"                     
 [18] "BUN, ISTAT"                                   
 [19] "BUN, Ser/Plas"                                
 [20] "Calcium, Ser/Plas"                            
 [21] "Chloride, ISTAT"                              
 [22] "Chloride, Ser/Plas"                           
 [23] "Chloride, Whole Bld"                          
 [24] "CO2 Arterial Total for POC"                   
 [25] "CO2, Ser/Plas"                                
 [26] "Creatinine, Ser/Plas"                         
 [27] "Creatinine,ISTAT"                             
 [28] "eGFR"                                         
 [29] "EOS, ABS"                                     
 [30] "Eos, ABS (man diff)"                          
 [31] "Eosinophil, Absolute"                         
 [32] "Eosinophils, ABS (man diff)"                  
 [33] "Globulin"                                     
 [34] "Glucose"                                      
 [35] "Glucose by Meter"                             
 [36] "Glucose, Non-fasting"                         
 [37] "Glucose, Ser/Plas"                            
 [38] "Glucose, Whole Blood"                         
 [39] "Glucose,ISTAT"                                
 [40] "HCO3"                                         
 [41] "HCO3 (a), ISTAT"                              
 [42] "HCO3 (v), ISTAT"                              
 [43] "HCO3, ISTAT"                                  
 [44] "Hct (Est)"                                    
 [45] "Hct, ISTAT"                                   
 [46] "HCT, POC"                                     
 [47] "Hct(Calc), ISTAT"                             
 [48] "Hematocrit"                                   
 [49] "Hematocrit (Manual Entry) See EMR for details"
 [50] "Hemoglobin"                                   
 [51] "HgB"                                          
 [52] "Hgb, calculated, POC"                         
 [53] "Hgb(Calc), ISTAT"                             
 [54] "INR"                                          
 [55] "INR, ISTAT"                                   
 [56] "Lactate, ISTAT"                               
 [57] "Lactate, Whole Bld"                           
 [58] "Lactic Acid"                                  
 [59] "LYM, ABS"                                     
 [60] "Lym, ABS (man diff)"                          
 [61] "Lymphocyte, Absolute"                         
 [62] "Lymphocytes, ABS (man diff)"                  
 [63] "Lymphocytes, Abs."                            
 [64] "MCH"                                          
 [65] "MONO, ABS"                                    
 [66] "Mono, ABS (man diff)"                         
 [67] "Monocyte, Absolute"                           
 [68] "Monocytes, ABS (man diff)"                    
 [69] "NEUT, ABS"                                    
 [70] "Neut, ABS (man diff)"                         
 [71] "Neut, ABS (Seg+Band) (man diff)"              
 [72] "Neutrophil, Absolute"                         
 [73] "Neutrophils, Absolute (Manual Diff)"          
 [74] "O2 Saturation (a)"                            
 [75] 

In [124]:
# check each lab in the list for reference units, low and high to see if consistent among similar lab names
lab_test <- labs0 %>% select(lab_name, ord_num_value, reference_unit, reference_low, reference_high, base_name)# %>%
#                         mutate(reference_unit = factor(reference_unit),
#                                reference_low = factor(reference_low),
#                                reference_high = factor(reference_high)) #%>% 
#                 drop_na(ord_num_value)
c=0
for (l in lab_names){
    c = c+1
    lab <- lab_test %>% filter(lab_name == l & ord_num_value != 9999999)
    print(l)
    print(lab %>% group_by(base_name) %>% count())
    print(summary(lab %>% select(ord_num_value, reference_unit, reference_low, reference_high)))
}

# options(repr.plot.width=7, repr.plot.height=7)
print(c)

# 'Albumin, Ser/Plas', 'Alk P\'TASE, Total, Ser/Plas', 'ALT (SGPT), Ser/Plas', 'AST (SGOT), Ser/Plas'
# 'Base Excess (vt)', 'Base Excess Arterial for POC', 'Base Excess, ISTAT'
# 'Calcium, Ser/Plas''CO2, Ser/Plas''eGFR''Globulin''Glucose'
# 'Glucose Urine''Glucose, urine''Glucose, Urine'
# 'MCH''Neut, ABS (man diff)''Platelets''Potassium''Protein, Total, Ser/Plas'
# 'RDW''Sodium, whole blood, ePOC'

[1] "Albumin, Ser/Plas"
# A tibble: 1 x 2
# Groups:   base_name [1]
  base_name     n
  <chr>     <int>
1 ALB       38273
 ord_num_value                               reference_unit  reference_low  
 Min.   :0.900   g/dL                               :38273   3.5    :38266  
 1st Qu.:3.000                                      :    0   3.9    :    4  
 Median :3.500                                      :    0   3.2    :    3  
 Mean   :3.472   /uL                                :    0          :    0  
 3rd Qu.:4.000   %                                  :    0   >60    :    0  
 Max.   :6.700   % (See scan or EMR data for detail):    0   >90    :    0  
                 (Other)                            :    0   (Other):    0  
 reference_high 
 5.0    :29710  
 5.2    : 8560  
 4.5    :    3  
        :    0  
 <0.055 :    0  
 <0.1   :    0  
 (Other):    0  
[1] "Alk P'TASE, Total, Ser/Plas"
# A tibble: 1 x 2
# Groups:   base_name [1]
  base_name     n
  <chr>     <int>
1 ALKP      

In [125]:
# update lab list, cut some labs - from TRIAGE
# OK to include 1 entry Glucose
# be careful with Lactate has 2 basename whole blood and none

# remove Potassium --> 1 entry, 7.4 weird
# remove Platelets --> all NA
# remove base_name of glucose urine --> all NA
# no longer here to remove: remove Basophils --> unit is %
# no longer here to remove: "pH" , "Ketone, urine", "Leukocyte Esterase, urine" # few, and all NA

lab_list <- c("RBC", "MCH", "MCHC", "MCV", "Calcium, Ser/Plas", "CO2, Ser/Plas",
               "Albumin, Ser/Plas", "ALT (SGPT), Ser/Plas", "AST (SGOT), Ser/Plas", "Alk P'TASE, Total, Ser/Plas",
               "Globulin", "Protein, Total, Ser/Plas",
               "Magnesium, Ser/Plas",  "Phosphorus, Ser/Plas", #14 ind
               
               "WBC", "WBC count", # WBC count is 1000*WBC
               "TROPONIN I", "Troponin I, POCT",
               "Total Bilirubin, Ser/Plas", "Total Bilirubin",
               "Platelet count", # remove Platelets
              
               "Hematocrit", "Hct, ISTAT", "Hct(Calc), ISTAT", "Hct (Est)", "HCT, POC", "Hematocrit (Manual Entry) See EMR for details",
               "Hemoglobin", "Hgb(Calc), ISTAT", "Hgb, calculated, POC", "HgB", 
               "Potassium, Ser/Plas", "Potassium, ISTAT",  "Potassium, Whole Bld", "Potassium, whole blood, ePOC", # remove Potassium
               "Sodium, Ser/Plas", "Sodium, ISTAT",  "Sodium, Whole Blood", 
               "Chloride, Ser/Plas",  "Chloride, ISTAT",  "Chloride, Whole Bld",
               "Creatinine, Ser/Plas", "Creatinine,ISTAT", 
              
               "Anion Gap", "Anion Gap, ISTAT",
               "Glucose, Ser/Plas", "Glucose,ISTAT", "Glucose, Whole Blood", "Glucose by Meter", "Glucose, Non-fasting",
               "BUN, Ser/Plas", "BUN, ISTAT", "Urea Nitrogen,Ser/Plas",
               "Neutrophil, Absolute", "NEUT, ABS", "Neut, ABS (Seg+Band) (man diff)", "Neutrophils, Absolute (Manual Diff)", "Neut, ABS (Seg+Band) (man diff)",
               "Basophil, Absolute", "BASOS, ABS", "Basophils, ABS (man diff)", "Baso, ABS (man diff)", # remove Basophils
               "Eosinophil, Absolute", "EOS, ABS", "Eosinophils, ABS (man diff)", "Eos, ABS (man diff)",
               "Lymphocyte, Absolute", "LYM, ABS", "Lymphocytes, ABS (man diff)", "Lym, ABS (man diff)", "Lymphocytes, Abs.",
                  # Lymphocytes, Abs. = 1000* the rest of lymphocytes
               "Monocyte, Absolute", "MONO, ABS", "Monocytes, ABS (man diff)", "Mono, ABS (man diff)",
               "Lactate, ISTAT", "Lactate, Whole Bld", "Lactic Acid",
               "Base Excess, ISTAT", "Base Excess (vt)", "Base Excess Arterial for POC",
              
                "HCO3", "HCO3 (a), ISTAT", "Bicarbonate, Art for POC", 
                "HCO3 (v), ISTAT", "HCO3, ISTAT", 
                "O2 Saturation (a)", "O2 Saturation, ISTAT", "Oxygen Saturation for POC", 
                "O2 Saturation (v)",  "O2 Saturation, ISTAT (Ven)",
                "pCO2 (a)", "pCO2 (a), ISTAT",  "PCO2, ISTAT", "Arterial pCO2 for POC",
                "pCO2 (v)", "PCO2 (v), ISTAT",
                "pH (a)", "PH (a), ISTAT", "pH by Meter", "Arterial pH for POC",
                "pH (v)", "PH (v), ISTAT", "PH, ISTAT", 
                "pO2 (a)", "PO2 (a), ISTAT",  "PO2, ISTAT", "Arterial pO2 for POC",
                "pO2 (v)", "PO2 (v), ISTAT", 
              
                "tCO2", "TCO2 (a), ISTAT","TCO2, ISTAT", "TCO2, (ISTAT)", "CO2 Arterial Total for POC", 
                "TCO2 (v), ISTAT", # 1 individual
                "INR", "INR, ISTAT", 
                "Prothrombin Time", "PT, ISTAT") # 33 more than 1
#               "pH", "Ketone, urine", "Leukocyte Esterase, urine") #"O2 Saturation, ISTAT (Oth)", "ctO2 (a)"
length(lab_list)

[1] 120

In [126]:
colnames(labs0)

[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "inpatient_data_id_coded" "admit_time"             
 [5] "label"                   "DBP"                    
 [7] "Pulse"                   "RR"                     
 [9] "SBP"                     "Temp"                   
[11] "order_id_coded"          "lab_name"               
[13] "base_name"               "ord_value"              
[15] "ord_num_value"           "reference_low"          
[17] "reference_high"          "reference_unit"         
[19] "result_in_range_yn"      "result_flag"            
[21] "result_time_utc"         "order_time_utc"         
[23] "taken_time_utc"

In [127]:
# sodium base_name is NA, so might not pick up. either collect is.na(base_name) or specify these lab_names
labs0 %>% filter(str_detect(lab_name, "Sodium")) %>% group_by(lab_name, base_name) %>% count() %>% distinct()

lab_name,base_name,n
<chr>,<chr>,<int>
"Sodium, ISTAT",NA,3188
"Sodium, Ser/Plas",NA,42093
"Sodium, Whole Blood",NA,330
"Sodium, whole blood, ePOC",NA,56


In [128]:
# Only need to remove glucose in urine, 1 potassium, 1 platelets, and base_name of urine glucose
removed_lab_names <- c("Platelets", "Potassium")
labs0 <- labs0 %>% filter(!lab_name %in% removed_lab_names, base_name != "GLUURN" | is.na(base_name)) %>%
                select(-c(taken_time_utc, order_time_utc)) %>%
#             select(anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_num_value, 
#                    result_flag, order_time_utc, taken_time_utc, result_time_utc) %>%
                rename(features = lab_name, values = ord_num_value, result_time = result_time_utc) %>% 
#             mutate(feature_type = "labs", result_flag = ifelse(result_flag=="",0,1)) %>% 
                drop_na(values) %>% distinct()
nrow(labs)

[1] 1367892

In [129]:
str_sort(setdiff(labs0$lab_name, lab_list)) # NONE
str_sort(setdiff(lab_list, labs0$features)) # ok
labs0 %>% filter(features %in% c('TCO2 (v), ISTAT')) # not there (only 1 person from the old dataset)
str_sort(unique(labs0$base_name)) # has NA

character(0)

[1] "Magnesium, Ser/Plas"  "MCHC"                 "MCV"                 
[4] "Phosphorus, Ser/Plas" "RBC"                  "TCO2 (v), ISTAT"

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label,DBP,Pulse,RR,SBP,Temp,⋯,features,base_name,ord_value,values,reference_low,reference_high,reference_unit,result_in_range_yn,result_flag,result_time
<chr>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<dbl>,⋯,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>


[1] "AG"      "AGAP"    "ALB"     "ALKP"    "ALT"     "AST"     "BASOAB" 
 [8] "BE"      "BUN"     "CA"      "CL"      "CO2"     "CR"      "EGFR"   
[15] "EOSAB"   "GLOB"    "GLU"     "HCO3"    "HCO3A"   "HCO3V"   "HCT"    
[22] "HGB"     "INR"     "K"       "LAC"     "LACWBL"  "LYMAB"   "MCH"    
[29] "MONOAB"  "NEUTAB"  "NEUTABS" "O2SATA"  "O2SATV"  "PCAGP"   "PCBUN"  
[36] "PCCL"    "PCO2A"   "PCO2V"   "PH"      "PHA"     "PHV"     "PLT"    
[43] "PO2A"    "PO2V"    "PT"      "RDW"     "TBIL"    "TCO2A"   "TNI"    
[50] "TP"      "WBC"     NA

In [130]:
# convert WBC count to WBC unit and Lymphocytes Abs. before combining
labs0 <- labs0 %>% mutate(values = ifelse(features == "WBC count", round(values/1000.0, 1), 
                                          ifelse(features == "Lymphocytes, Abs.", round(values/1000.0, 3), values))) 

In [131]:
# in range: Y or blank 
labs0 %>% count(result_in_range_yn) %>% arrange(desc(result_in_range_yn))
# normal as blanks
labs0 %>% count(result_flag) %>% arrange(desc(n))
head(labs0 %>% count(values) %>% arrange())
tail(labs0 %>% count(values) %>% arrange())

result_in_range_yn,n
<fct>,<int>
Y,23524
,1344314


result_flag,n
<fct>,<int>
,920593
High,219631
Low,210634
High Panic,8504
Low Panic,8457
Abnormal,19


,values,n
,<dbl>,<int>
1,-30.0,2
2,-29.8,1
3,-29.0,12
4,-28.0,14
5,-27.7,1
6,-27.0,22


,values,n
,<dbl>,<int>
6110,6307,1
6111,6589,1
6112,6673,1
6113,6729,1
6114,9655,1
6115,9999999,15666


In [132]:
labs0 %>% group_by(features, base_name) %>% count() %>% distinct()

features,base_name,n
<chr>,<chr>,<int>
"Albumin, Ser/Plas",ALB,38315
"Alk P'TASE, Total, Ser/Plas",ALKP,38129
"ALT (SGPT), Ser/Plas",ALT,38040
Anion Gap,AG,41927
"Anion Gap, ISTAT",AGAP,2509
"Anion Gap, ISTAT",PCAGP,155
Arterial pCO2 for POC,PCO2A,12
Arterial pH for POC,PHA,12
Arterial pO2 for POC,PO2A,12


In [133]:
str_sort(unique(labs0$features))

[1] "Albumin, Ser/Plas"                            
  [2] "Alk P'TASE, Total, Ser/Plas"                  
  [3] "ALT (SGPT), Ser/Plas"                         
  [4] "Anion Gap"                                    
  [5] "Anion Gap, ISTAT"                             
  [6] "Arterial pCO2 for POC"                        
  [7] "Arterial pH for POC"                          
  [8] "Arterial pO2 for POC"                         
  [9] "AST (SGOT), Ser/Plas"                         
 [10] "Base Excess (vt)"                             
 [11] "Base Excess Arterial for POC"                 
 [12] "Base Excess, ISTAT"                           
 [13] "Baso, ABS (man diff)"                         
 [14] "Basophil, Absolute"                           
 [15] "Basophils, ABS (man diff)"                    
 [16] "BASOS, ABS"                                   
 [17] "Bicarbonate, Art for POC"                     
 [18] "BUN, ISTAT"                                   
 [19] "BUN, Ser/Plas"                                
 [20] "Calcium, Ser/Plas"                            
 [21] "Chloride, ISTAT"                              
 [22] "Chloride, Ser/Plas"                           
 [23] "Chloride, Whole Bld"                          
 [24] "CO2 Arterial Total for POC"                   
 [25] "CO2, Ser/Plas"                                
 [26] "Creatinine, Ser/Plas"                         
 [27] "Creatinine,ISTAT"                             
 [28] "eGFR"                                         
 [29] "EOS, ABS"                                     
 [30] "Eos, ABS (man diff)"                          
 [31] "Eosinophil, Absolute"                         
 [32] "Eosinophils, ABS (man diff)"                  
 [33] "Globulin"                                     
 [34] "Glucose"                                      
 [35] "Glucose by Meter"                             
 [36] "Glucose, Non-fasting"                         
 [37] "Glucose, Ser/Plas"                            
 [38] "Glucose, Whole Blood"                         
 [39] "Glucose,ISTAT"                                
 [40] "HCO3"                                         
 [41] "HCO3 (a), ISTAT"                              
 [42] "HCO3 (v), ISTAT"                              
 [43] "HCO3, ISTAT"                                  
 [44] "Hct (Est)"                                    
 [45] "Hct, ISTAT"                                   
 [46] "HCT, POC"                                     
 [47] "Hct(Calc), ISTAT"                             
 [48] "Hematocrit"                                   
 [49] "Hematocrit (Manual Entry) See EMR for details"
 [50] "Hemoglobin"                                   
 [51] "HgB"                                          
 [52] "Hgb, calculated, POC"                         
 [53] "Hgb(Calc), ISTAT"                             
 [54] "INR"                                          
 [55] "INR, ISTAT"                                   
 [56] "Lactate, ISTAT"                               
 [57] "Lactate, Whole Bld"                           
 [58] "Lactic Acid"                                  
 [59] "LYM, ABS"                                     
 [60] "Lym, ABS (man diff)"                          
 [61] "Lymphocyte, Absolute"                         
 [62] "Lymphocytes, ABS (man diff)"                  
 [63] "Lymphocytes, Abs."                            
 [64] "MCH"                                          
 [65] "MONO, ABS"                                    
 [66] "Mono, ABS (man diff)"                         
 [67] "Monocyte, Absolute"                           
 [68] "Monocytes, ABS (man diff)"                    
 [69] "NEUT, ABS"                                    
 [70] "Neut, ABS (man diff)"                         
 [71] "Neut, ABS (Seg+Band) (man diff)"              
 [72] "Neutrophil, Absolute"                         
 [73] "Neutrophils, Absolute (Manual Diff)"          
 [74] "O2 Saturation (a)"                            
 [75] 

In [134]:
# ISTAT seems all A, so put pH in A
# combine same labs and change the lab name in the data
# could use base_name except for LACTATE has 2 different base_names
# Glucose lab_name as base_name as GLU and UGLU. exclude glucose here, only 1 entry anyways
# New: Glucose, Sodium ePOC, 'Neut, ABS (man diff)' and ALB - RDW

ALB = 'Albumin, Ser/Plas'
ALK = 'Alk P\'TASE, Total, Ser/Plas'
ALT = 'ALT (SGPT), Ser/Plas'
AST = 'AST (SGOT), Ser/Plas'

Ca = 'Calcium, Ser/Plas'
CO2 = 'CO2, Ser/Plas'
eGFR = 'eGFR' # as char
Glob = 'Globulin'

MCH = 'MCH' # as char
TProtein = 'Protein, Total, Ser/Plas'
RDW = 'RDW' # as char

Platelet = c("Platelet count") # remove platelets 
TBili = c("Total Bilirubin", "Total Bilirubin, Ser/Plas")
Trop = c("TROPONIN I", "Troponin I, POCT")
WBC = c("WBC", "WBC count") # WBC count /1000 7

Hct = c("Hct (Est)", "Hct, ISTAT", "HCT, POC", "Hct(Calc), ISTAT", "Hematocrit",    
        "Hematocrit (Manual Entry) See EMR for details")
Hgb = c("Hemoglobin", "HgB", "Hgb, calculated, POC", "Hgb(Calc), ISTAT")
K = c("Potassium, ISTAT",  "Potassium, Ser/Plas", "Potassium, Whole Bld", "Potassium, whole blood, ePOC") # remove Potassium
Na = c("Sodium, ISTAT",  "Sodium, Ser/Plas", "Sodium, Whole Blood", 'Sodium, whole blood, ePOC') # ePOC is new
Cl = c("Chloride, ISTAT",  "Chloride, Ser/Plas",  "Chloride, Whole Bld")
Cr = c("Creatinine, Ser/Plas", "Creatinine,ISTAT") #22

AnionGap = c("Anion Gap", "Anion Gap, ISTAT") 
Glucose = c('Glucose', "Glucose by Meter", "Glucose, Non-fasting", "Glucose, Ser/Plas", "Glucose, Whole Blood", "Glucose,ISTAT")
BUN = c("BUN, Ser/Plas", "BUN, ISTAT", "Urea Nitrogen,Ser/Plas") #10
Neut = c("NEUT, ABS", 'Neut, ABS (man diff)', "Neut, ABS (Seg+Band) (man diff)", "Neutrophil, Absolute",  "Neutrophils, Absolute (Manual Diff)")
Basos = c("Basophils, ABS (man diff)", "Basophil, Absolute", "Baso, ABS (man diff)", "BASOS, ABS")
Eos = c("EOS, ABS", "Eosinophils, ABS (man diff)", "Eosinophil, Absolute", "Eos, ABS (man diff)")
Lymp = c("LYM, ABS", "Lymphocytes, ABS (man diff)", "Lymphocyte, Absolute", "Lym, ABS (man diff)", "Lymphocytes, Abs.")
Mono = c("MONO, ABS", "Monocytes, ABS (man diff)", "Monocyte, Absolute", "Mono, ABS (man diff)")
Lactate = c("Lactate, ISTAT", "Lactate, Whole Bld", "Lactic Acid")
Base = c("Base Excess (vt)", "Base Excess Arterial for POC", "Base Excess, ISTAT") #28

HCO3_a = c("HCO3", "HCO3 (a), ISTAT", "Bicarbonate, Art for POC") # HCO3 is A based on base name
HCO3_v = c("HCO3 (v), ISTAT", "HCO3, ISTAT") 
O2sat_a = c("O2 Saturation (a)", "O2 Saturation, ISTAT", "Oxygen Saturation for POC")
O2sat_v = c("O2 Saturation (v)",  "O2 Saturation, ISTAT (Ven)")
pCO2_a = c("pCO2 (a)", "pCO2 (a), ISTAT",  "PCO2, ISTAT", "Arterial pCO2 for POC")
pCO2_v = c("pCO2 (v)", "PCO2 (v), ISTAT") 
pH_a = c("pH (a)", "PH (a), ISTAT", "pH by Meter", "Arterial pH for POC", "PH, ISTAT")
pH_v = c ("pH (v)", "PH (v), ISTAT") # new: move "PH, ISTAT" to arterial
PO2_a = c("pO2 (a)", "PO2 (a), ISTAT",  "PO2, ISTAT", "Arterial pO2 for POC")
PO2_v = c ("pO2 (v)", "PO2 (v), ISTAT") #29

TCO2_a = c("tCO2", "TCO2 (a), ISTAT", "TCO2, (ISTAT)", "TCO2, ISTAT", "CO2 Arterial Total for POC")
INR = c("INR", "INR, ISTAT")
PT = c("Prothrombin Time", "PT, ISTAT") #9

labs <- labs0 %>% 
        mutate(features = 
            ifelse(features %in% Platelet, "Platelet", ifelse(features %in% TBili, "TBili", 
            ifelse(features %in% Trop, "Trop", ifelse(features %in% WBC, "WBC",
            ifelse(features %in% Hct, "Hct", ifelse(features %in% Hgb, "Hgb", 
            ifelse(features %in% K, "K", ifelse(features %in% Na, "Na",
            ifelse(features %in% Cl, "Cl", ifelse(features %in% Cr, "Cr",
            ifelse(features %in% AnionGap, "AnionGap", ifelse(features %in% Glucose, "Glucose",
            ifelse(features %in% BUN, "BUN", ifelse(features %in% Neut, "Neut",    
            ifelse(features %in% Basos, "Basos", ifelse(features %in% Eos, "Eos",      
            ifelse(features %in% Lymp, "Lymp", ifelse(features %in% Mono, "Mono",    
            ifelse(features %in% Lactate, "Lactate",  ifelse(features %in% Base, "Base", 
            ifelse(features %in% HCO3_a, "HCO3_a", ifelse(features %in% HCO3_v, "HCO3_v", 
            ifelse(features %in% O2sat_a, "O2sat_a", ifelse(features %in% O2sat_v, "O2sat_v",
            ifelse(features %in% pCO2_a, "pCO2_a", ifelse(features %in% pCO2_v, "pCO2_v",
            ifelse(features %in% pH_a, "pH_a", ifelse(features %in% pH_v, "pH_v",      
            ifelse(features %in% PO2_a, "PO2_a", ifelse(features %in% PO2_v, "PO2_v",
            ifelse(features %in% TCO2_a, "TCO2_a", ifelse(features %in% INR, "INR",
            ifelse(features %in% PT, "PT", ifelse(features %in% ALB, "ALB",
            ifelse(features %in% ALK, "ALK", ifelse(features %in% ALT, "ALT",
            ifelse(features %in% AST, "AST", ifelse(features %in% TProtein, "TProtein",                                     
            ifelse(features %in% Ca, "Ca", ifelse(features %in% CO2, "CO2",                                        
            ifelse(features %in% Glob, "Glob", as.character(features))))))))))))))))))))))))))))))))))))))))))) %>%
        distinct() 


In [135]:
nrow(labs) # 1367892 vs 1367838
# total 48 lab_name but 56 base_name
nrow(labs %>% select(anon_id, pat_enc_csn_id_coded) %>% distinct()) #41627
# labs %>% select(base_name, features) %>% distinct() %>% arrange(features, base_name) 
labs %>% group_by(features, base_name) %>% count(sort=TRUE) %>% arrange(features, base_name) 

[1] 1367838

[1] 41627

features,base_name,n
<chr>,<chr>,<int>
ALB,ALB,38315
ALK,ALKP,38129
ALT,ALT,38040
AnionGap,AG,41927
AnionGap,AGAP,2509
AnionGap,PCAGP,155
AST,AST,37997
Base,BE,20497
Basos,BASOAB,36201


### Process the 9999999 values
- check ord_value

In [136]:
# there are no NA, but only 15668 of 9999999 values
value99 <- labs %>% filter(values==9999999) 
nrow(value99)
value99 %>% group_by(ord_value) %>% count() %>% arrange(-n)

[1] 15666

ord_value,n
<chr>,<int>
>60,5328
<0.017,4560
<0.1,1540
>90,1366
<10,552
N/A,409
<0.2,375
>500,242
<0.30,226


In [137]:
value99 %>% filter(ord_value %in% c("N/A", "hi") | str_detect(ord_value, 'CANCELLED')) %>% count(ord_value, sort=TRUE)
value99 %>% filter(!grepl('>|<', ord_value)) %>% count(ord_value, sort=TRUE)

value99 %>% filter(ord_value %in% c("N/A", "hi") | str_detect(ord_value, 'CANCELLED')) %>% count(features, sort=TRUE)

ord_value,n
<chr>,<int>
N/A,409
CANCELLED: Specimen Clotted.,2
CANCELLED: Note:,1
CANCELLED: Clerical Error,1
CANCELLED: Duplicate Order,1
CANCELLED: Inappropriate Specimen/Container.,1
hi,1


ord_value,n
<chr>,<int>
N/A,409
CANCELLED: Specimen Clotted.,2
CANCELLED: Note:,1
CANCELLED: Clerical Error,1
CANCELLED: Duplicate Order,1
CANCELLED: Inappropriate Specimen/Container.,1
hi,1


features,n
<chr>,<int>
Platelet,22
Base,21
Basos,21
Eos,21
Lymp,21
Mono,21
Neut,21
MCH,18
RDW,18


In [138]:
# number of cancelled, NA, or 1 hi observations under ord_value
nrow(labs %>% filter(values==9999999, !grepl('>|<', ord_value)) %>% mutate(ord_value = factor(ord_value))) # %>% group_by(ord_value) %>% count() %>% arrange(-n)

[1] 416

In [140]:
# revmove those without > or < in ord_value (416), work with this set and then do it on the main labs
# take the upper/lower bounds for the 9999999 values and add 1% to it
value99 <- value99 %>% filter(grepl('>|<', ord_value)) %>% #mutate(ord_value = as.character(ord_value)) %>%
            mutate(values = 1.01 * as.double(gsub(paste(c(">", "<"), collapse = "|"),"", ord_value))) #%>%
#             drop_na(values) # %>% distinct() # same

nrow(value99) #15250
summary(value99$values)

[1] 15250

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
 -30.300    0.017   10.100   46.964   60.600 3535.000 

In [141]:
head(value99)

,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label,DBP,Pulse,RR,SBP,Temp,⋯,features,base_name,ord_value,values,reference_low,reference_high,reference_unit,result_in_range_yn,result_flag,result_time
,<chr>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<dbl>,⋯,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>
1,JCdab3d0,131094116026,16013755,2015-06-29 01:26:00+00:00,1,52,76,22,76,36.00,⋯,TCO2_a,TCO2A,<5,5.05,23,27,mmol/L,,Low Panic,2015-06-28 20:03:00+00:00
2,JCd81b89,131084822387,15250783,2015-06-03 22:06:00+00:00,1,73,105,24,146,37.00,⋯,pCO2_a,PCO2A,>130.0,131.30,35.0,45.0,mmHg,,High Panic,2015-06-03 20:06:00+00:00
3,JCd08ac9,131102861750,16164115,2015-07-03 18:21:00+00:00,1,46,102,18,80,33.80,⋯,TCO2_a,TCO2A,<5,5.05,23,27,mmol/L,,Low Panic,2015-07-03 15:49:00+00:00
4,JCe9719e,131179594045,18773115,2016-02-20 00:09:00+00:00,1,96,89,24,157,36.20,⋯,TCO2_a,TCO2A,>50,50.50,23,27,mmol/L,,High Panic,2016-02-19 19:09:00+00:00
5,JCe21eb0,131120616215,16787756,2015-10-26 03:44:00+00:00,1,75,80,16,116,36.85,⋯,Lactate,LACWBL,>15.0,15.15,,<2.0,mmol/L,,High,2015-10-26 01:41:00+00:00
6,JCce9b8f,131117935744,16717626,2015-09-04 11:45:00+00:00,1,76,132,22,143,36.85,⋯,TCO2_a,TCO2A,<5,5.05,23,27,mmol/L,,Low Panic,2015-09-04 07:38:00+00:00


In [165]:
# remove only the observations (where values = 999999 and do not have < or > in ord_value) from process labs
# could do 1.01 or 0.99 but more complicated
grepl('>|<', ">60")
labs99 <- labs %>% filter(!(values==9999999 & !grepl('>|<', ord_value))) %>%
            mutate(values = as.double(gsub(paste(c(">", "<"), collapse = "|"),"", ord_value)),
                   feature_type = "labs") 
nrow(labs) - nrow(labs99) # 1367892 - 1367476
nrow(labs99) # 1367476 vs 1367422
summary(labs99$values)
colnames(labs99)

[1] TRUE

[1] 416

[1] 1367422

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  -30.0     3.4    12.9    39.2    38.3 11900.0 

[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "inpatient_data_id_coded" "admit_time"             
 [5] "label"                   "DBP"                    
 [7] "Pulse"                   "RR"                     
 [9] "SBP"                     "Temp"                   
[11] "order_id_coded"          "features"               
[13] "base_name"               "ord_value"              
[15] "values"                  "reference_low"          
[17] "reference_high"          "reference_unit"         
[19] "result_in_range_yn"      "result_flag"            
[21] "result_time"             "feature_type"

In [166]:
labs99 <- labs99 %>% select(-c('DBP','Pulse','RR','SBP','Temp', 'order_id_coded'))
colnames(labs99)

[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "inpatient_data_id_coded" "admit_time"             
 [5] "label"                   "features"               
 [7] "base_name"               "ord_value"              
 [9] "values"                  "reference_low"          
[11] "reference_high"          "reference_unit"         
[13] "result_in_range_yn"      "result_flag"            
[15] "result_time"             "feature_type"

In [167]:
write.csv(labs99, file.path(featuredir, "2_4_coh3_labs.csv"), row.names=FALSE)

### Keep this cohort information:

In [156]:
cohort_with_labs <- labs99 %>% select(anon_id, pat_enc_csn_id_coded, inpatient_data_id_coded, admit_time, label) %>% distinct()
nrow(cohort_with_labs)

cohort_with_labs %>% group_by(label) %>% summarise(count = n(), percent = round(100*count/nrow(cohort_with_labs),2))

[1] 41627

label,count,percent
<int>,<int>,<dbl>
0,35868,86.17
1,5759,13.83


In [155]:
write.csv(cohort_with_labs, file.path(cohortdir, "cohort3L_withlabs.csv"), row.names=FALSE)